In [20]:
%%time
import re
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt

from sklearn.svm import OneClassSVM
from sklearn.datasets import make_blobs
from numpy import quantile, where, random
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import LocalOutlierFactor
from sklearn.ensemble import IsolationForest

CPU times: total: 0 ns
Wall time: 0 ns


In [2]:
import re

log_entry = 'nova-api.log.2017-05-16_13:53:08 2017-05-16 06:24:48.767 25746 ERROR nova.osapi_compute.wsgi.server [req-743d5247-0dd8-4e57-8aa3-bcfbd09b9d6d 113d3a99c3da401fbd62cc2caa5b96d2 54fadb412c4e40cdbaed9335e4c35a9e - - -] 10.11.10.1 "DELETE /v2/54fadb412c4e40cdbaed9335e4c35a9e/servers HTTP/1.1" status: 202 len: 733 time: 0.5041740'

# Extracting information using regular expressions
# pattern = r'(.*?)(\d{4}-\d{2}-\d{2}\s+\d{2}:\d{2}:\d{2}\.\d+)\s+(\d+)\s+(\w+)\s+(\S+)\s+\[([^\]]+)\]\s+(\S+)\s+"(\S+)\s+(\S+)\s+(\S+)"\s+status:\s+(\d+)\s+len:\s+(\d+)\s+time:\s+(\S+)'
# pattern = r'(.*?)(\d{4}-\d{2}-\d{2}\s+\d{2}:\d{2}:\d{2}\.\d+)\s+(\d+)\s+(\w+)\s+(\S+)\s+\[([^\]]+)\]\s+(\S+)\s+"(\S+)\s+(\S+)\s+(\S+)"\s+status:\s+(\d+)\s+len:\s+(\d+)\s+time:\s+([\d.]+)'
pattern = r'(.*?)(\d{4}-\d{2}-\d{2}\s+\d{2}:\d{2}:\d{2}\.\d+)\s+(\d+)\s+(\w+)\s+(\S+)\s+\[([^\]]+)\]\s+(\S+)\s+"(\S+)\s+(\S+)\s+(\S+)"\s+status:\s+(\d+)\s+len:\s+(\d+)\s+time:\s+([\d.]+)'

matches = re.match(pattern, log_entry)

# Parsing the extracted information
log_filename = matches.group(1)
timestamp = matches.group(2)
process_id = matches.group(3)
log_level = matches.group(4)
component = matches.group(5)
request_info = matches.group(6)
ip = matches.group(7)
http_method = matches.group(8)
url = matches.group(9)
http_protocol = matches.group(10)
status_code = matches.group(11)
response_length = matches.group(12)
response_time = matches.group(13)

# Print the parsed information
print("Log Filename:", log_filename)
print("Timestamp:", timestamp)
print("Process ID:", process_id)
print("Log Level:", log_level)
print("Component:", component)
print("Request Info:", request_info)
print("IP:", ip)
print("HTTP Method:", http_method)
print("URL:", url)
print("HTTP Protocol:", http_protocol)
print("Status Code:", status_code)
print("Response Length:", response_length)
print("Response Time:", response_time)


Log Filename: nova-api.log.2017-05-16_13:53:08 
Timestamp: 2017-05-16 06:24:48.767
Process ID: 25746
Log Level: ERROR
Component: nova.osapi_compute.wsgi.server
Request Info: req-743d5247-0dd8-4e57-8aa3-bcfbd09b9d6d 113d3a99c3da401fbd62cc2caa5b96d2 54fadb412c4e40cdbaed9335e4c35a9e - - -
IP: 10.11.10.1
HTTP Method: DELETE
URL: /v2/54fadb412c4e40cdbaed9335e4c35a9e/servers
HTTP Protocol: HTTP/1.1
Status Code: 202
Response Length: 733
Response Time: 0.5041740


In [3]:
def parse_nova_api_log(log_entry):
    pattern = r'(.*?)(\d{4}-\d{2}-\d{2}\s+\d{2}:\d{2}:\d{2}\.\d+)\s+(\d+)\s+(\w+)\s+(\S+)\s+\[([^\]]+)\]\s+(\S+)\s+"(\S+)\s+(\S+)\s+(\S+)"\s+status:\s+(\d+)\s+len:\s+(\d+)\s+time:\s+([\d.]+)'
    match = re.match(pattern, log_entry)
    if match:
        return {
    # Parsing the extracted information
            'log_filename' : matches.group(1),
            'timestamp' : matches.group(2),
            'process_id' : matches.group(3),
            'log_level' : matches.group(4),
            'component' : matches.group(5),
            'request_info' : matches.group(6),
            'ip_address' : matches.group(7),
            'http_method' : matches.group(8),
            'url' : matches.group(9),
            'http_protocol' : matches.group(10),
            'status_code' : matches.group(11),
            'response_length' : matches.group(12),
            'response_time' : matches.group(13)
        },match
    else:
        return None, match


import re

def parse_log_entry(log_entry):
    pattern = re.compile(
    r"(?P<log_name>.*?)(?:\.\d+)?(?:\.\d{4}-\d{2}-\d{2}_\d{2}:\d{2}:\d{2})?\s(?P<timestamp>\d{4}-\d{2}-\d{2}\s\d{2}:\d{2}:\d{2}\.\d{3})\s(?P<process_id>\d+)\s(?P<log_level>[A-Z]+)\s(?P<module>[\w\.]+)\s\[(?P<request_id>req-[a-f0-9\-]+).*\]\s(?P<message>.+)")
    match = pattern.match(log_entry)
    if match:
        log_data = match.groupdict()
        return {
            'log_name': log_data['log_name'],
            'timestamp': log_data['timestamp'],
            'process_id': log_data['process_id'],
            'log_level': log_data['log_level'],
            'Module': log_data["module"],
            'Request': log_data["request_id"],
            'Message':log_data["message"]
        }, match
    else:
        return None, match

In [4]:
log_entry2 = 'nova-api.log.1.2017-05-16_13:53:08 2017-05-16 06:24:48.767 25746 WARNING nova.osapi_compute.wsgi.server [req-743d5247-0dd8-4e57-8aa3-bcfbd09b9d6d 113d3a99c3da401fbd62cc2caa5b96d2 54fadb412c4e40cdbaed9335e4c35a9e - - -] 10.11.10.1 "POST /v2/54fadb412c4e40cdbaed9335e4c35a9e/servers HTTP/1.1" status: 202 len: 733 time: 0.5041740'
# log_entry3 = 'nova-api.log.2017-05-14_21:27:04 2017-05-14 19:39:22.564 25743 INFO nova.api.openstack.compute.server_external_events [req-f2e0dc94-6f72-42e9-9b92-39dde4ae1618 f7b8d1f1d4d44643b07fa10ca7d021fb e9746973ac574c6b8a9e8857f56a7608 - - -] Creating event network-vif-plugged:95fcdc5e-3159-4e86-968a-13a2a06e9e89 for instance 3edec1e4-9678-4a3a-a21b-a145a4ee5e61'
parse_nova_api_log(log_entry2)

({'log_filename': 'nova-api.log.2017-05-16_13:53:08 ',
  'timestamp': '2017-05-16 06:24:48.767',
  'process_id': '25746',
  'log_level': 'ERROR',
  'component': 'nova.osapi_compute.wsgi.server',
  'request_info': 'req-743d5247-0dd8-4e57-8aa3-bcfbd09b9d6d 113d3a99c3da401fbd62cc2caa5b96d2 54fadb412c4e40cdbaed9335e4c35a9e - - -',
  'ip_address': '10.11.10.1',
  'http_method': 'DELETE',
  'url': '/v2/54fadb412c4e40cdbaed9335e4c35a9e/servers',
  'http_protocol': 'HTTP/1.1',
  'status_code': '202',
  'response_length': '733',
  'response_time': '0.5041740'},
 <re.Match object; span=(0, 327), match='nova-api.log.1.2017-05-16_13:53:08 2017-05-16 06:>)

In [5]:
log_compute = "nova-compute.log.2017-05-14_21:27:09 2017-05-14 19:39:02.007 2931 INFO nova.virt.libvirt.driver [req-e285b551-587f-4c1d-8eba-dceb2673637f 113d3a99c3da401fbd62cc2caa5b96d2 54fadb412c4e40cdbaed9335e4c35a9e - - -] [instance: 3edec1e4-9678-4a3a-a21b-a145a4ee5e61] Creating image"
parse_log_entry(log_compute)

({'log_name': 'nova-compute.log',
  'timestamp': '2017-05-14 19:39:02.007',
  'process_id': '2931',
  'log_level': 'INFO',
  'Module': 'nova.virt.libvirt.driver',
  'Request': 'req-e285b551-587f-4c1d-8eba-dceb2673637f',
  'Message': 'Creating image'},
 <re.Match object; span=(0, 274), match='nova-compute.log.2017-05-14_21:27:09 2017-05-14 1>)

In [6]:
log_scheduler = "nova-scheduler.log.2017-05-14_21:27:04 2017-05-14 19:40:50.128 25998 INFO nova.scheduler.host_manager [req-7c8dcf36-20e9-477c-a826-26f875a07800 - - - - -] Successfully synced instances from host 'cp-1.slowvm1.tcloud-pg0.utah.cloudlab.us'."
parse_log_entry(log_scheduler)

({'log_name': 'nova-scheduler.log',
  'timestamp': '2017-05-14 19:40:50.128',
  'process_id': '25998',
  'log_level': 'INFO',
  'Module': 'nova.scheduler.host_manager',
  'Request': 'req-7c8dcf36-20e9-477c-a826-26f875a07800',
  'Message': "Successfully synced instances from host 'cp-1.slowvm1.tcloud-pg0.utah.cloudlab.us'."},
 <re.Match object; span=(0, 238), match="nova-scheduler.log.2017-05-14_21:27:04 2017-05-14>)

In [7]:
import re

# log_pattern_api = r'(nova-api\.log\.\d{4}-\d{2}-\d{2}_\d{2}:\d{2}:\d{2})\s(\d{4}-\d{2}-\d{2}\s\d{2}:\d{2}:\d{2}\.\d{3})\s(\d+)\s(\w+)\s([\w.]+)\s\[(.*?)\]\s(.*?)\sstatus:\s(\d+)\slen:\s(\d+)\stime:\s([\d.]+)'
# log_pattern_compute = r'(nova-compute\.log\.\d{4}-\d{2}-\d{2}_\d{2}:\d{2}:\d{2})\s(\d{4}-\d{2}-\d{2}\s\d{2}:\d{2}:\d{2}\.\d{3})\s(\d+)\s(\w+)\s([\w.]+)\s\[(.*?)\]\s\[(.*?)\]\s(.*)'
# log_pattern_scheduler = r'(nova-scheduler\.log\.\d{4}-\d{2}-\d{2}_\d{2}:\d{2}:\d{2})\s(\d{4}-\d{2}-\d{2}\s\d{2}:\d{2}:\d{2}\.\d{3})\s(\d+)\s(\w+)\s([\w.]+)\s\[(.*?)\]\s(.*)'

log_files = ['./OpenStack/openstack_abnormal.log', './OpenStack/openstack_normal1.log', './OpenStack/openstack_normal2.log']
parsed_logs_api = []
parsed_logs_compute = []
parsed_logs_scheduler = []

# Open the log file and parse the log entries
for log_file in log_files:
    with open(log_file, 'r') as f:
        for line in f:
            api_log, api_match = parse_nova_api_log(line)
            compute_log, compute_match = parse_log_entry(line)
            scheduler_log, scheduler_match = parse_log_entry(line)
            anomaly_value = -1 if 'abnormal' in log_file else 1
            
            if api_log:
                log_data = list(api_match.groups())
                log_data.append(anomaly_value)
                parsed_logs_api.append(log_data)
                
            elif compute_log and "nova-compute.log" in line:
                log_data = list(compute_match.groups())
                log_data.append(anomaly_value)
                parsed_logs_compute.append(log_data)
                
            elif scheduler_log and "nova-scheduler.log" in line:
                log_data = list(scheduler_match.groups())
                log_data.append(anomaly_value)
                parsed_logs_scheduler.append(log_data)
#             elif not api_match and 'nova-api.log' in line:
#                 print(f"Line does not match any log pattern: {line}")


In [8]:
print(len(parsed_logs_api))
print(len(parsed_logs_compute))
print(len(parsed_logs_scheduler))
print("total = " , len(parsed_logs_api) +len(parsed_logs_compute) +len(parsed_logs_scheduler))

98733
97636
866
total =  197235


In [9]:
import pandas as pd
df_api = pd.DataFrame(parsed_logs_api, columns=['log_filename', 'timestamp', 'process_id', 'log_level', 'component', 'request_info', 'ip_address', 'http_method', 'url', 'http_protocol', 'status_code', 'response_length', 'response_time','Anomaly'])
df_api = df_api.sample(frac = 1).reset_index(drop=True)
df_api.to_csv('nova-api.csv', index=False)

df_compute = pd.DataFrame(parsed_logs_compute, columns=['log_name', 'timestamp', 'process_id', 'log_level', 'module', 'request', 'message','Anomaly'])
df_compute = df_compute.sample(frac = 1).reset_index(drop=True)
df_compute.to_csv('nova-compute-api.csv', index=False)

df_scheduler = pd.DataFrame(parsed_logs_scheduler, columns=['log_name', 'timestamp', 'process_id', 'log_level', 'module', 'request', 'message','Anomaly'])
df_scheduler = df_scheduler.sample(frac = 1).reset_index(drop=True)
df_scheduler.to_csv('nova-scheduler-api.csv', index=False)

In [19]:
df_api.head()

,log_filename,timestamp,process_id,log_level,component,request_info,ip_address,http_method,url,http_protocol,status_code,response_length,response_time,Anomaly
0,nova-api.log.1.2017-05-16_13:53:08,2017-05-16 05:43:06.160,25746,INFO,nova.osapi_compute.wsgi.server,req-1f3e9d96-f4fb-4ebf-9831-c564f299d524 113d3...,10.11.10.1,GET,/v2/54fadb412c4e40cdbaed9335e4c35a9e/servers/d...,HTTP/1.1,200,1893,0.2813430,1
1,nova-api.log.1.2017-05-16_13:53:08,2017-05-16 02:13:31.746,25746,INFO,nova.osapi_compute.wsgi.server,req-fe6511f7-4d4b-46de-b458-b86ba1f4d4e7 113d3...,10.11.10.1,GET,/v2/54fadb412c4e40cdbaed9335e4c35a9e/servers/d...,HTTP/1.1,200,1892,0.2634771,1
2,nova-api.log.1.2017-05-17_12:02:19,2017-05-16 15:16:46.736,25749,INFO,nova.osapi_compute.wsgi.server,req-e5f69ff7-e5c6-458a-adad-b116c817a31d 113d3...,10.11.10.1,GET,/v2/54fadb412c4e40cdbaed9335e4c35a9e/servers/d...,HTTP/1.1,200,1759,0.2716548,1
3,nova-api.log.1.2017-05-16_13:53:08,2017-05-16 04:05:34.119,25784,INFO,nova.metadata.wsgi.server,-,"10.11.22.220,10.11.10.1",GET,/openstack/2013-10-17/meta_data.json,HTTP/1.1,200,967,0.0011399,1
4,nova-api.log.1.2017-05-16_13:53:08,2017-05-16 00:37:11.485,25746,INFO,nova.osapi_compute.wsgi.server,req-ae647483-d1f3-4513-82d5-ad3559cc3947 113d3...,10.11.10.1,GET,/v2/54fadb412c4e40cdbaed9335e4c35a9e/servers/d...,HTTP/1.1,200,1893,0.2438290,1


In [20]:
df_compute.head()

,log_name,timestamp,process_id,log_level,module,request,message,Anomaly
0,nova-compute.log,2017-05-16 05:19:35.290,2931,INFO,nova.compute.manager,req-3ea4052c-895d-4b64-9e2d-04d64c4d94ab,VM Resumed (Lifecycle Event),1
1,nova-compute.log,2017-05-17 09:46:51.051,2931,INFO,nova.virt.libvirt.imagecache,req-addc1839-2ed5-4778-b57e-5854eb7b8b09,image 0673dd71-34c5-4fbb-86c4-40623fbe45b4 at ...,1
2,nova-compute.log,2017-05-16 01:03:04.927,2931,INFO,nova.compute.manager,req-3ea4052c-895d-4b64-9e2d-04d64c4d94ab,VM Resumed (Lifecycle Event),1
3,nova-compute.log,2017-05-16 02:22:08.090,2931,INFO,nova.compute.manager,req-3e14378c-a48d-4a17-9fad-ba29c6bb6960,Took 1.03 seconds to destroy the instance on t...,1
4,nova-compute.log,2017-05-17 05:22:50.546,2931,INFO,nova.virt.libvirt.imagecache,req-addc1839-2ed5-4778-b57e-5854eb7b8b09,Active base files: /var/lib/nova/instances/_ba...,1


In [21]:
df_scheduler.head()

,log_name,timestamp,process_id,log_level,module,request,message,Anomaly
0,nova-scheduler.log,2017-05-17 05:42:33.210,25998,INFO,nova.scheduler.host_manager,req-d23551d5-9152-4d05-a0de-d08a3c9b5500,The instance sync for host 'cp-1.slowvm1.tclou...,1
1,nova-scheduler.log,2017-05-17 00:05:45.151,25998,INFO,nova.scheduler.host_manager,req-5b5c9432-a37a-4cc3-8afa-3c02f82f8a90,The instance sync for host 'cp-1.slowvm1.tclou...,1
2,nova-scheduler.log,2017-05-17 06:41:50.365,25998,INFO,nova.scheduler.host_manager,req-4f225652-1be6-4266-b668-491611cd02b1,Successfully synced instances from host 'cp-1....,1
3,nova-scheduler.log,2017-05-16 06:13:47.256,25998,INFO,nova.scheduler.host_manager,req-0f043983-07cb-4167-8efc-fcacbeb6815e,Successfully synced instances from host 'cp-1....,1
4,nova-scheduler.log,2017-05-16 00:09:04.153,25998,INFO,nova.scheduler.host_manager,req-702e2535-33f8-4590-bd45-7c7fbd5b8c1f,The instance sync for host 'cp-1.slowvm1.tclou...,1


In [2]:
df_api = pd.read_csv("./nova-api.csv")
df_compute = pd.read_csv("./nova-compute-api.csv")
df_scheduler = pd.read_csv("./nova-scheduler-api.csv")

In [3]:
# Convert 'timestamp' to datetime if it's not already in that format

df_api['timestamp'] = pd.to_datetime(df_api['timestamp'])  
df_compute['timestamp'] = pd.to_datetime(df_compute['timestamp'])  
df_scheduler['timestamp'] = pd.to_datetime(df_scheduler['timestamp'])  

# df_api with one-classSVM and LOF

In [4]:
X_api = df_api[['process_id','log_level','component','ip_address','http_method','status_code', 'response_length', 'response_time']] 

In [5]:
X_api['log_duration'] = df_api['timestamp'].diff().dt.total_seconds().fillna(0)

C:\Users\test\AppData\Local\Temp\ipykernel_11552\3585568061.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_api['log_duration'] = df_api['timestamp'].diff().dt.total_seconds().fillna(0)


In [6]:
X_api.head()

,process_id,log_level,component,ip_address,http_method,status_code,response_length,response_time,log_duration
0,25746,INFO,nova.osapi_compute.wsgi.server,10.11.10.1,GET,200,1893,0.281343,0.000
1,25746,INFO,nova.osapi_compute.wsgi.server,10.11.10.1,GET,200,1892,0.263477,-12574.414
2,25749,INFO,nova.osapi_compute.wsgi.server,10.11.10.1,GET,200,1759,0.271655,46994.990
3,25784,INFO,nova.metadata.wsgi.server,"10.11.22.220,10.11.10.1",GET,200,967,0.001140,-40272.617
4,25746,INFO,nova.osapi_compute.wsgi.server,10.11.10.1,GET,200,1893,0.243829,-12502.634


In [7]:
from sklearn.preprocessing import LabelEncoder

# Create an instance of LabelEncoder
label_encoder = LabelEncoder()
string_columns = ['log_level', 'component', 'ip_address', 'http_method']
# Iterate over each string column and apply label encoding
for column in string_columns:
    encoded_feature = label_encoder.fit_transform(X_api[column])
    X_api[column] = encoded_feature

C:\Users\test\AppData\Local\Temp\ipykernel_11552\1578764275.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_api[column] = encoded_feature


In [8]:
X_api.head()

,process_id,log_level,component,ip_address,http_method,status_code,response_length,response_time,log_duration
0,25746,0,1,0,1,200,1893,0.281343,0.000
1,25746,0,1,0,1,200,1892,0.263477,-12574.414
2,25749,0,1,0,1,200,1759,0.271655,46994.990
3,25784,0,0,468,1,200,967,0.001140,-40272.617
4,25746,0,1,0,1,200,1893,0.243829,-12502.634


In [9]:
scaler = StandardScaler()

# Assuming df_api is your DataFrame
column_to_keep = 'ip_address'  # Specify the column you want to keep unchanged

columns_to_scale = X_api.columns.difference([column_to_keep])  # Get the remaining columns to scale

scaler = StandardScaler()
scaled_data = scaler.fit_transform(X_api[columns_to_scale])  # Scale the selected columns

X_scaled = X_api.copy()  # Create a copy of the original DataFrame
X_scaled[columns_to_scale] = scaled_data  # Assign the scaled values back to the DataFrame

X_api = X_scaled

X_api.head()

,process_id,log_level,component,ip_address,http_method,status_code,response_length,response_time,log_duration
0,-0.609344,0.0,0.510367,0,-0.146175,-0.207931,0.324639,0.450409,-0.000008
1,-0.609344,0.0,0.510367,0,-0.146175,-0.207931,0.323923,0.280255,-0.154211
2,-0.417685,0.0,0.510367,0,-0.146175,-0.207931,0.228746,0.358139,0.576304
3,1.818334,0.0,-1.959373,468,-0.146175,-0.207931,-0.338025,-2.218245,-0.493882
4,-0.609344,0.0,0.510367,0,-0.146175,-0.207931,0.324639,0.093126,-0.153331


In [10]:
svm = OneClassSVM(kernel='rbf', gamma=0.001, nu=0.03)
print(svm)

OneClassSVM(gamma=0.001, nu=0.03)


In [11]:
svm.fit(X_api)
pred = svm.predict(X_api)

In [12]:
X_api['anomaly_svm'] = pred

In [13]:
X_api.head(20)

,process_id,log_level,component,ip_address,http_method,status_code,response_length,response_time,log_duration,anomaly_svm
0,-0.609344,0.0,0.510367,0,-0.146175,-0.207931,0.324639,0.450409,-0.000008,1
1,-0.609344,0.0,0.510367,0,-0.146175,-0.207931,0.323923,0.280255,-0.154211,1
2,-0.417685,0.0,0.510367,0,-0.146175,-0.207931,0.228746,0.358139,0.576304,1
3,1.818334,0.0,-1.959373,468,-0.146175,-0.207931,-0.338025,-2.218245,-0.493882,1
4,-0.609344,0.0,0.510367,0,-0.146175,-0.207931,0.324639,0.093126,-0.153331,1
5,-0.417685,0.0,0.510367,0,-0.146175,-0.207931,0.341098,0.399714,1.162104,1
6,-0.417685,0.0,0.510367,0,-0.146175,-0.207931,0.324639,0.218700,-0.246116,1
7,-0.609344,0.0,0.510367,0,-0.146175,-0.207931,0.323923,0.378873,-0.829894,1
8,-0.417685,0.0,0.510367,0,-0.146175,-0.207931,0.324639,0.369856,0.861771,1
9,-0.417685,0.0,0.510367,0,-0.146175,-0.207931,0.324639,0.306892,-0.143120,1


In [14]:


clf = LocalOutlierFactor(n_neighbors=20, contamination=0.1)

y_pred = clf.fit_predict(X_api)

In [15]:
X_api['anomaly_LOF'] = y_pred

In [16]:
X_api.head(20)

,process_id,log_level,component,ip_address,http_method,status_code,response_length,response_time,log_duration,anomaly_svm,anomaly_LOF
0,-0.609344,0.0,0.510367,0,-0.146175,-0.207931,0.324639,0.450409,-0.000008,1,1
1,-0.609344,0.0,0.510367,0,-0.146175,-0.207931,0.323923,0.280255,-0.154211,1,1
2,-0.417685,0.0,0.510367,0,-0.146175,-0.207931,0.228746,0.358139,0.576304,1,1
3,1.818334,0.0,-1.959373,468,-0.146175,-0.207931,-0.338025,-2.218245,-0.493882,1,1
4,-0.609344,0.0,0.510367,0,-0.146175,-0.207931,0.324639,0.093126,-0.153331,1,1
5,-0.417685,0.0,0.510367,0,-0.146175,-0.207931,0.341098,0.399714,1.162104,1,1
6,-0.417685,0.0,0.510367,0,-0.146175,-0.207931,0.324639,0.218700,-0.246116,1,1
7,-0.609344,0.0,0.510367,0,-0.146175,-0.207931,0.323923,0.378873,-0.829894,1,1
8,-0.417685,0.0,0.510367,0,-0.146175,-0.207931,0.324639,0.369856,0.861771,1,1
9,-0.417685,0.0,0.510367,0,-0.146175,-0.207931,0.324639,0.306892,-0.143120,1,1


In [17]:
X_api['real_anomaly'] = df_api['Anomaly']

In [18]:
X_api.head(20)

,process_id,log_level,component,ip_address,http_method,status_code,response_length,response_time,log_duration,anomaly_svm,anomaly_LOF,real_anomaly
0,-0.609344,0.0,0.510367,0,-0.146175,-0.207931,0.324639,0.450409,-0.000008,1,1,1
1,-0.609344,0.0,0.510367,0,-0.146175,-0.207931,0.323923,0.280255,-0.154211,1,1,1
2,-0.417685,0.0,0.510367,0,-0.146175,-0.207931,0.228746,0.358139,0.576304,1,1,1
3,1.818334,0.0,-1.959373,468,-0.146175,-0.207931,-0.338025,-2.218245,-0.493882,1,1,1
4,-0.609344,0.0,0.510367,0,-0.146175,-0.207931,0.324639,0.093126,-0.153331,1,1,1
5,-0.417685,0.0,0.510367,0,-0.146175,-0.207931,0.341098,0.399714,1.162104,1,1,1
6,-0.417685,0.0,0.510367,0,-0.146175,-0.207931,0.324639,0.218700,-0.246116,1,1,1
7,-0.609344,0.0,0.510367,0,-0.146175,-0.207931,0.323923,0.378873,-0.829894,1,1,1
8,-0.417685,0.0,0.510367,0,-0.146175,-0.207931,0.324639,0.369856,0.861771,1,1,1
9,-0.417685,0.0,0.510367,0,-0.146175,-0.207931,0.324639,0.306892,-0.143120,1,1,1


In [44]:
model=IsolationForest(n_estimators=50,max_samples='auto',contamination='auto',random_state=42)

model.fit(X_api)
X_api['anomaly_isolation'] = model.predict(X_api)


In [32]:
X_api.head(20)

,process_id,log_level,component,ip_address,http_method,status_code,response_length,response_time,log_duration,anomaly_svm,anomaly_LOF,real_anomaly,anomaly_isolation
0,-0.609344,0.0,0.510367,0,-0.146175,-0.207931,0.324639,0.450409,-0.000008,1,1,1,1
1,-0.609344,0.0,0.510367,0,-0.146175,-0.207931,0.323923,0.280255,-0.154211,1,1,1,1
2,-0.417685,0.0,0.510367,0,-0.146175,-0.207931,0.228746,0.358139,0.576304,1,1,1,1
3,1.818334,0.0,-1.959373,468,-0.146175,-0.207931,-0.338025,-2.218245,-0.493882,1,1,1,-1
4,-0.609344,0.0,0.510367,0,-0.146175,-0.207931,0.324639,0.093126,-0.153331,1,1,1,1
5,-0.417685,0.0,0.510367,0,-0.146175,-0.207931,0.341098,0.399714,1.162104,1,1,1,1
6,-0.417685,0.0,0.510367,0,-0.146175,-0.207931,0.324639,0.218700,-0.246116,1,1,1,1
7,-0.609344,0.0,0.510367,0,-0.146175,-0.207931,0.323923,0.378873,-0.829894,1,1,1,1
8,-0.417685,0.0,0.510367,0,-0.146175,-0.207931,0.324639,0.369856,0.861771,1,1,1,1
9,-0.417685,0.0,0.510367,0,-0.146175,-0.207931,0.324639,0.306892,-0.143120,1,1,1,1


In [23]:
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score, confusion_matrix

In [24]:
f1_svm = f1_score(X_api['real_anomaly'],X_api['anomaly_svm'])
acc_svm = accuracy_score(X_api['real_anomaly'],X_api['anomaly_svm'])
roc_svm = roc_auc_score(X_api['real_anomaly'],X_api['anomaly_svm'])
#sen_svm, spec_svm = get_sensitivity_specificity(X_api['real_anomaly'],X_api['anomaly_svm'])
met_svm = {
           'f1_score': f1_svm,
           'accuracy': acc_svm,
           'roc_score': roc_svm,
           #'sensitivity': sen_svm,
           #'specificity': spec_svm
          }

In [25]:
f1_lof = f1_score(X_api['real_anomaly'],X_api['anomaly_LOF'])
acc_lof = accuracy_score(X_api['real_anomaly'],X_api['anomaly_LOF'])
roc_lof = roc_auc_score(X_api['real_anomaly'],X_api['anomaly_LOF'])
#sen_svm, spec_svm = get_sensitivity_specificity(X_api['real_anomaly'],X_api['anomaly_svm'])
met_lof = {
           'f1_score': f1_lof,
           'accuracy': acc_lof,
           'roc_score': roc_lof,
           #'sensitivity': sen_svm,
           #'specificity': spec_svm
          }

In [46]:
f1_iso = f1_score(X_api['real_anomaly'],X_api['anomaly_isolation'])
acc_iso = accuracy_score(X_api['real_anomaly'],X_api['anomaly_isolation'])
roc_iso = roc_auc_score(X_api['real_anomaly'],X_api['anomaly_isolation'])
#sen_svm, spec_svm = get_sensitivity_specificity(X_api['real_anomaly'],X_api['anomaly_svm'])
met_isolation = {
           'f1_score': f1_iso,
           'accuracy': acc_iso,
           'roc_score': roc_iso,
           #'sensitivity': sen_svm,
           #'specificity': spec_svm
          }

In [47]:
metrics = {#'isolated_forest': met_iso,
           #'kmeans': met_kmeans,
           'svm': met_svm,
           'lof': met_lof,
           'isolation': met_isolation
          }

In [48]:
metrics_df_api = pd.DataFrame.from_dict(metrics)

In [49]:
metrics_df_api

,svm,lof,isolation
f1_score,0.935847,0.902883,0.819709
accuracy,0.879746,0.824739,0.709489
roc_score,0.498596,0.503013,0.622295


# df-compute oneclassSVM  and LOF

In [40]:
# List of columns you want to analyze
columns_to_analyze = ['process_id', 'log_level', 'module','request','message']  # Replace with the actual column names

for column in columns_to_analyze:
    # Count the number of unique values and store them in a variable
    unique_values_count = df_compute[column].nunique()

    # Get the count of each unique value and store it in a variable
    value_counts = df_compute[column].value_counts()

    # Print the number of unique values for the current column
    print("Number of unique values in", column, ":", unique_values_count)

    # Iterate over the value_counts Series and print each unique value and its count
    for value, count in value_counts.items():
        print(value, ":", count)

    # Add a separator line between columns
    print('-' * 30)
print(len(df_compute))

Number of unique values in process_id : 1
2931 : 97636
------------------------------
Number of unique values in log_level : 3
INFO : 94512
ERROR : 8
------------------------------
Number of unique values in module : 6
nova.virt.libvirt.imagecache : 45298
nova.compute.manager : 22648
nova.compute.claims : 16520
nova.compute.resource_tracker : 6956
nova.virt.libvirt.driver : 6206
oslo_service.periodic_task : 8
------------------------------
Number of unique values in request : 4134
req-addc1839-2ed5-4778-b57e-5854eb7b8b09 : 52349
req-3ea4052c-895d-4b64-9e2d-04d64c4d94ab : 12248
req-d44b9064-57a4-409c-abf7-51263f5fd562 : 11
req-b3de0124-0f4c-4e5b-b3d9-10d6aada54b7 : 11
req-2aa70835-ffb1-435f-b5dc-4d622b7f411c : 11
req-4fe0286d-a5e2-4490-aad0-26f4e932248d : 11
req-a0e11c92-f9ff-431f-bd9b-a94ead49ada9 : 11
req-bd3f37c5-6d8f-4416-ae8a-447ca058f25b : 11
req-c4aacaae-343e-47c5-93c4-26e5981857ad : 11
req-07840fda-86d3-4660-9e26-574e4340cd93 : 11
req-2e56ace2-1a0a-4d94-977f-4ee0b625630e : 11
re

req-4413a05e-b790-4dac-a5dc-5ed8d8e714fe : 11
req-2816c362-624c-40fb-a5de-c1405758f338 : 11
req-02d253ae-e84e-43d4-8c35-174e5fbde154 : 11
req-e7d48ce5-83b0-4795-a2a0-ad4db73628c0 : 11
req-63ca6712-dd77-4d13-aced-4dd9a335a6aa : 11
req-d94b52ff-a7b5-455a-8c79-a0d21545b1f9 : 11
req-b1fdb5fa-22ef-44a0-9de0-5ac0db710d2b : 11
req-6b3dd8c9-6b49-4923-afb6-516b9332c223 : 11
req-b2de89ec-3b08-42ca-8273-20a7b12f86c3 : 11
req-f7a8bb77-7f4e-45c9-b3b2-650667f5d95c : 11
req-74e14a91-16b0-44f4-99a5-a02651d9fe98 : 11
req-afe8e7f4-a039-4673-a904-a8961dfaa0e4 : 11
req-d38f479d-9bb9-4276-9688-52607e8fd350 : 11
req-d6143bd4-9784-4e95-a0c8-83fba00530b6 : 11
req-1df0cdc5-cfa3-4e19-ac93-49f3bf4f2dd1 : 11
req-cc356996-ce56-4a2e-9f98-0d7d2235c209 : 11
req-d1935b90-eda0-45c5-8783-78910aba11bf : 11
req-f7f2110f-a839-47ed-a7f7-f1de9b27f911 : 11
req-051050a0-f260-45b8-b9d8-f6391d50a501 : 11
req-922d2a99-f4c6-4d5d-b532-c7b5ad0466bc : 11
req-0230a2b1-2837-4ed8-94f1-cc2074a9f2a6 : 11
req-dba947a4-238a-4587-b7a3-14c513

req-4f221acb-b5a6-42a0-92e7-ab0887919a97 : 11
req-1adefd78-49da-49ae-80f6-80ce186f7b09 : 11
req-f11733cb-7cf5-4fa3-a831-3c12104fc1c8 : 11
req-b22ba49a-b129-42c5-b805-9f714307cb48 : 11
req-064f7c9a-f66e-4eaa-9382-aaf5b212a759 : 11
req-83c58a06-5dad-4f63-a13e-d32bda2579e1 : 11
req-9d0bd61a-ca74-489f-860d-97e1971bf52e : 11
req-8e004b41-198f-4e5f-9f2c-bec97f9ba894 : 11
req-70eca47d-221b-41e9-9e52-ed46a2088dc7 : 11
req-3be787e9-4610-4053-a290-c941e30abd05 : 11
req-8127d8e4-dafb-47b3-bde9-960ac62693f4 : 11
req-fa7d5a4a-593d-4ded-a563-0aa3811dde1e : 11
req-e8b785ea-1681-4d69-b2b4-f33005cc90f0 : 11
req-a68139aa-ff56-434f-883a-3089ceea46c1 : 11
req-34056e1a-46c5-426e-8fec-e560fdd17077 : 11
req-ae270e99-8227-482a-8796-e07b44562a3c : 11
req-d0c258be-96e0-49d4-af24-55b4aa5d4f6c : 11
req-d7faccbd-0586-4afa-a06e-bc43ed656dc6 : 11
req-bd9f0209-d130-4b2d-93fa-554b048143d0 : 11
req-f317f84d-c397-40bb-805a-a026dae8b1fa : 11
req-5d16de5a-5de9-4fd2-b294-51b041551451 : 11
req-8022bb02-4625-4a7a-bc2a-42aaaf

req-5e5ed336-424a-4823-a646-8081bfea235c : 5
req-a071f889-ffad-4128-8e3e-51f054a25b0d : 5
req-33e6140b-c291-4f76-bc31-bfca0376ce5b : 5
req-5ded64ca-40d4-480e-9567-2bd1668f274d : 5
req-05490367-be02-4d74-b4e0-55341ab37cc4 : 5
req-f8bb4df5-1c7b-4aad-8e9d-c1b7c2467db4 : 5
req-795fcd03-9a08-414a-8c44-468de5712eee : 5
req-5f0698b7-22c3-4619-ae9e-8700ad919b5d : 5
req-c91e46f8-7517-4231-b2e6-81470027c097 : 5
req-414ffdc3-df7c-4a64-8a72-1a5a95bb7f8e : 5
req-88f99b14-3852-4fd5-9c4a-51c514998291 : 5
req-084f6519-bc97-47ae-af0f-fa3719efffd4 : 5
req-11b1b347-4f46-415e-a6ce-e7157e280e20 : 5
req-d1b61a84-5273-4a99-8da1-2cad7d716ea7 : 5
req-7757307d-842f-47ab-92c7-4e9e087243d9 : 5
req-58d4a20d-5c92-41ea-ab59-e6a915495668 : 5
req-5ce21e4f-4b15-4682-8508-b1fa9e1946b8 : 5
req-0abbfab5-a0bb-45e3-9f95-305bf129ff52 : 5
req-7ee39b8e-f38c-4bee-ad3b-dfcab0db6f7b : 5
req-105d6488-8799-495b-b102-5c0dc4c090e2 : 5
req-592ad91b-d1fa-4b7d-bee4-aff0e3590026 : 5
req-ab16c4cf-b159-4719-aa7e-3c8926f77c35 : 5
req-f780b7

req-d9d236e2-525c-4133-81ac-f439669bcb37 : 5
req-13e2a103-f916-4606-ad5a-0c2d686dcb21 : 5
req-94091576-43f1-4a09-bf8c-4a6822fb1be7 : 5
req-44cef58c-010c-4eba-a012-3974245c9931 : 5
req-01da99ae-3285-4499-b2ba-cc705abd3e9c : 5
req-2ca33f68-ca9c-469a-bbb1-11ea0e006180 : 5
req-182403a1-e50d-41a9-8b3a-5750ecc6df97 : 5
req-711af880-2add-4785-909f-5aaf598df083 : 5
req-c349f07d-3a85-457a-875a-0563a8847e54 : 5
req-a2c4ebdd-fef9-462d-8b9a-8cb35a00218c : 5
req-214a93e6-e4f6-446b-abb1-cea1e78bb6db : 5
req-57eb0a5f-431b-4414-a25f-f0ba26ef66a7 : 5
req-386cffc8-6a8c-4a4b-ae53-f509a4c3ff5e : 5
req-2f0c68ef-6cf3-40df-b71c-c9ec50a6f21a : 5
req-ff6284d1-1200-408c-8f2e-18435eef5347 : 5
req-ed82e332-dffe-41d0-8c0e-7862d200dec0 : 5
req-71edf51c-5296-436f-94d6-c879ca10697a : 5
req-5cca4a6e-879f-4427-bb7e-1c004edbb2ee : 5
req-9d08264e-20e9-4dc3-a1fe-bfa84ae3d496 : 5
req-1c85876d-ad87-46fa-8267-9156c38fa0dc : 5
req-2703e9ba-a675-4965-8baf-d3d52bbcf0ba : 5
req-5113d6b4-5028-4a0d-b25b-cede192416ba : 5
req-a2193a

...

Deleting instance files /var/lib/nova/instances/432cf4d3-f459-4da4-8992-71cc78d019f1_del : 1
Deletion of /var/lib/nova/instances/d0c4724b-7a63-4f94-829d-2e60f6f8373a_del complete : 1
Deletion of /var/lib/nova/instances/c721eb1b-0346-44d9-ac87-56fb20413d07_del complete : 1
Deleting instance files /var/lib/nova/instances/9f02b411-dd3f-42b8-8cbd-e657d7a2b49c_del : 1
Deletion of /var/lib/nova/instances/6f4820cc-594d-42a7-a69d-aca00f0b13d9_del complete : 1
Deleting instance files /var/lib/nova/instances/d116384e-af44-4cd4-b3ac-fcbe1e8620ba_del : 1
Deleting instance files /var/lib/nova/instances/be2ea012-d088-4cad-b539-9f906551bb84_del : 1
Deleting instance files /var/lib/nova/instances/be67189c-fcdd-4b01-a8d0-68a0652dc0cc_del : 1
Deleting instance files /var/lib/nova/instances/27274c0e-481b-4097-9295-b17ddc36e22b_del : 1
Deletion of /var/lib/nova/instances/2be32db2-d6bd-4d74-8ddc-0c34d0ed9a06_del complete : 1
Deletion of /var/lib/nova/instances/6e6d4952-0005-4a96-b8dc-6d278dbd40a2_del compl

Deleting instance files /var/lib/nova/instances/96938bfb-ff16-4711-b91f-f00279699650_del : 1
Deleting instance files /var/lib/nova/instances/154dd8e4-ff52-41a6-b320-bf16651bb0fa_del : 1
Deleting instance files /var/lib/nova/instances/5abf034c-26bb-487b-a2db-6c4b068c7f88_del : 1
Deleting instance files /var/lib/nova/instances/8f77d04e-ac04-459e-a13e-8429c901e1f1_del : 1
Deleting instance files /var/lib/nova/instances/25e3c974-a5ab-4f6d-8d4c-bd9d9eb9ee49_del : 1
Deletion of /var/lib/nova/instances/f1d7ad04-445d-4973-a194-3526ca488c25_del complete : 1
Deletion of /var/lib/nova/instances/47ccfb96-57d8-4e2e-aedd-899bc2102d88_del complete : 1
Deleting instance files /var/lib/nova/instances/d908871a-7a2f-4d40-b8f6-ac6b74e88019_del : 1
Deletion of /var/lib/nova/instances/064735e0-539f-408a-aa53-0b52075309a5_del complete : 1
Deleting instance files /var/lib/nova/instances/56d2a77c-0f89-41f6-b510-7586c98d1f60_del : 1
Deleting instance files /var/lib/nova/instances/0e893750-3717-47a4-bd6c-de29e42

Deleting instance files /var/lib/nova/instances/56afd89e-a700-4df1-98c4-f4a770acf450_del : 1
Deleting instance files /var/lib/nova/instances/f6f7c7e3-8e24-456b-8f99-b1e4a02a9592_del : 1
Took 21.37 seconds to build instance. : 1
Deleting instance files /var/lib/nova/instances/4fcff866-1f71-4fa9-8d07-6073e1fe9512_del : 1
Deleting instance files /var/lib/nova/instances/b9a24c3b-b802-44f2-9bb1-40bab6fae9de_del : 1
Deleting instance files /var/lib/nova/instances/4802dac8-a373-4daf-9cff-3c17f2397fb0_del : 1
Deletion of /var/lib/nova/instances/963c1d4b-94f9-4381-b245-40d58184ddaf_del complete : 1
Deleting instance files /var/lib/nova/instances/c1903023-e563-4df1-af0d-d747948b0d82_del : 1
Deletion of /var/lib/nova/instances/322a408a-1634-4800-b3d5-b62e7a9cc2ab_del complete : 1
Deletion of /var/lib/nova/instances/cc2b4649-9cf0-4dfc-a2f6-26bb55fd0e93_del complete : 1
Deletion of /var/lib/nova/instances/d0463ab4-d5fa-4519-960e-c93d3f17843e_del complete : 1
Deleting instance files /var/lib/nova/in

Deletion of /var/lib/nova/instances/46c7c600-a921-4c6a-92f0-de5fd5ddf5da_del complete : 1
Deletion of /var/lib/nova/instances/c1903023-e563-4df1-af0d-d747948b0d82_del complete : 1
Deleting instance files /var/lib/nova/instances/9a1d8c51-be5d-4a87-b59b-0be0262ec487_del : 1
Deletion of /var/lib/nova/instances/16190e72-3347-45ba-b998-127fd3813e1a_del complete : 1
Deleting instance files /var/lib/nova/instances/a015cf14-84bb-4156-a48d-7c4824ac7a9d_del : 1
Deleting instance files /var/lib/nova/instances/3881b20a-1898-4384-a049-2c6ba9f62fc7_del : 1
Deletion of /var/lib/nova/instances/2bb30d7d-d530-482f-9852-8af3cf6d7800_del complete : 1
Took 10.94 seconds to build instance. : 1
Deletion of /var/lib/nova/instances/61b2178e-2a89-4083-a8ff-e1648c7950fc_del complete : 1
Deleting instance files /var/lib/nova/instances/93975623-68c6-4a2f-a1a5-e82dfaa5f683_del : 1
Deletion of /var/lib/nova/instances/a454fc56-8a10-4508-b9f5-86bd228facb8_del complete : 1
Deletion of /var/lib/nova/instances/9ad07043-2

Deleting instance files /var/lib/nova/instances/e200d7bd-cd44-43fc-82e9-dc688d30eb0e_del : 1
Deleting instance files /var/lib/nova/instances/e9811b29-81ee-4639-bcb2-2554c4aa8ed3_del : 1
Deletion of /var/lib/nova/instances/002afded-8d0b-49f2-bab7-07e8ff79eac4_del complete : 1
Deletion of /var/lib/nova/instances/168bed28-81d3-4582-8a97-b2a4c55ab349_del complete : 1
Deleting instance files /var/lib/nova/instances/62d7723f-fc42-4610-aeba-9d8f8d3db54a_del : 1
Deleting instance files /var/lib/nova/instances/6416ee7c-dad6-4863-840d-af534554fa3c_del : 1
Deleting instance files /var/lib/nova/instances/b0bf8e9b-5988-438a-95c3-1ec7f0000434_del : 1
Deletion of /var/lib/nova/instances/b93fafb2-046c-4c23-a778-fba49a964d4c_del complete : 1
Deletion of /var/lib/nova/instances/265403c9-39fb-46dd-8ff6-34ba364a5881_del complete : 1
Deleting instance files /var/lib/nova/instances/daf38630-e1ca-42b1-8b71-4df078076a47_del : 1
Deleting instance files /var/lib/nova/instances/1dbcdff4-2bc2-431f-9444-b2e855d74d

In [4]:
X_compute = df_compute.loc[:,['log_level','module','request']] 

In [5]:
X_compute['log_duration'] = df_compute['timestamp'].diff().dt.total_seconds().fillna(0)
X_compute['message_length'] = df_compute['message'].apply(len)

In [6]:
X_compute.head()

,log_level,module,request,log_duration,message_length
0,INFO,nova.compute.manager,req-3ea4052c-895d-4b64-9e2d-04d64c4d94ab,0.000,28
1,INFO,nova.virt.libvirt.imagecache,req-addc1839-2ed5-4778-b57e-5854eb7b8b09,102435.761,128
2,INFO,nova.compute.manager,req-3ea4052c-895d-4b64-9e2d-04d64c4d94ab,-117826.124,28
3,INFO,nova.compute.manager,req-3e14378c-a48d-4a17-9fad-ba29c6bb6960,4743.163,60
4,INFO,nova.virt.libvirt.imagecache,req-addc1839-2ed5-4778-b57e-5854eb7b8b09,97242.456,89


In [10]:
# Create an instance of LabelEncoder
label_encoder = LabelEncoder()
string_columns = ['log_level', 'module', 'request']
# Iterate over each string column and apply label encoding
for column in string_columns:
    encoded_feature = label_encoder.fit_transform(X_compute[column])
    X_compute[column] = encoded_feature

In [11]:
X_compute.head()

,log_level,module,request,log_duration,message_length
0,1,1,1048,0.000,28
1,1,4,2824,102435.761,128
2,1,1,1048,-117826.124,28
3,1,1,1036,4743.163,60
4,1,4,2824,97242.456,89


In [12]:
scaler = StandardScaler()

# Assuming df_api is your DataFrame
column_to_keep = 'ip_address'  # Specify the column you want to keep unchanged

columns_to_scale = X_compute.columns.difference([column_to_keep])  # Get the remaining columns to scale

scaler = StandardScaler()
scaled_data = scaler.fit_transform(X_compute[columns_to_scale])  # Scale the selected columns

X_scaled = X_compute.copy()  # Create a copy of the original DataFrame
X_scaled[columns_to_scale] = scaled_data  # Assign the scaled values back to the DataFrame

X_compute = X_scaled

X_compute.head()

,log_level,module,request,log_duration,message_length
0,-0.180846,-0.872904,-1.410575,5.416904e-07,-1.130151
1,-0.180846,0.969520,0.520977,1.228456e+00,0.739208
2,-0.180846,-0.872904,-1.410575,-1.413023e+00,-1.130151
3,-0.180846,-0.872904,-1.423626,5.688266e-02,-0.531956
4,-0.180846,0.969520,0.520977,1.166175e+00,0.010158


In [14]:
svm_compute = OneClassSVM(kernel='rbf', gamma=0.001, nu=0.03)
print(svm_compute)

OneClassSVM(gamma=0.001, nu=0.03)


In [15]:
svm_compute.fit(X_compute)
pred = svm_compute.predict(X_compute)

In [16]:
X_compute['anomaly_svm'] = pred

In [17]:
X_compute.head()

,log_level,module,request,log_duration,message_length,anomaly_svm
0,-0.180846,-0.872904,-1.410575,5.416904e-07,-1.130151,1
1,-0.180846,0.969520,0.520977,1.228456e+00,0.739208,1
2,-0.180846,-0.872904,-1.410575,-1.413023e+00,-1.130151,1
3,-0.180846,-0.872904,-1.423626,5.688266e-02,-0.531956,1
4,-0.180846,0.969520,0.520977,1.166175e+00,0.010158,1


In [19]:
clf_compute = LocalOutlierFactor(n_neighbors=20, contamination=0.1)

y_pred = clf_compute.fit_predict(X_compute)

In [20]:
X_compute['anomaly_LOF'] = y_pred

In [21]:
X_compute.head()

,log_level,module,request,log_duration,message_length,anomaly_svm,anomaly_LOF
0,-0.180846,-0.872904,-1.410575,5.416904e-07,-1.130151,1,1
1,-0.180846,0.969520,0.520977,1.228456e+00,0.739208,1,1
2,-0.180846,-0.872904,-1.410575,-1.413023e+00,-1.130151,1,-1
3,-0.180846,-0.872904,-1.423626,5.688266e-02,-0.531956,1,1
4,-0.180846,0.969520,0.520977,1.166175e+00,0.010158,1,1


In [22]:
X_compute['real_anomaly'] = df_compute['Anomaly']
X_compute.head()

,log_level,module,request,log_duration,message_length,anomaly_svm,anomaly_LOF,real_anomaly
0,-0.180846,-0.872904,-1.410575,5.416904e-07,-1.130151,1,1,1
1,-0.180846,0.969520,0.520977,1.228456e+00,0.739208,1,1,1
2,-0.180846,-0.872904,-1.410575,-1.413023e+00,-1.130151,1,-1,1
3,-0.180846,-0.872904,-1.423626,5.688266e-02,-0.531956,1,1,1
4,-0.180846,0.969520,0.520977,1.166175e+00,0.010158,1,1,1


In [57]:
f1_svm = f1_score(X_compute['real_anomaly'],X_compute['anomaly_svm'])
acc_svm = accuracy_score(X_compute['real_anomaly'],X_compute['anomaly_svm'])
roc_svm = roc_auc_score(X_compute['real_anomaly'],X_compute['anomaly_svm'])
#sen_svm, spec_svm = get_sensitivity_specificity(X_api['real_anomaly'],X_api['anomaly_svm'])
met_svm = {
           'f1_score': f1_svm,
           'accuracy': acc_svm,
           'roc_score': roc_svm,
           #'sensitivity': sen_svm,
           #'specificity': spec_svm
          }

In [58]:
f1_lof = f1_score(X_compute['real_anomaly'],X_compute['anomaly_LOF'])
acc_lof = accuracy_score(X_compute['real_anomaly'],X_compute['anomaly_LOF'])
roc_lof = roc_auc_score(X_compute['real_anomaly'],X_compute['anomaly_LOF'])
#sen_svm, spec_svm = get_sensitivity_specificity(X_api['real_anomaly'],X_api['anomaly_svm'])
met_lof = {
           'f1_score': f1_lof,
           'accuracy': acc_lof,
           'roc_score': roc_lof,
           #'sensitivity': sen_svm,
           #'specificity': spec_svm
          }

In [59]:
metrics = {#'isolated_forest': met_iso,
           #'kmeans': met_kmeans,
           'svm': met_svm,
           'lof': met_lof
          }

In [60]:
metrics_df_api = pd.DataFrame.from_dict(metrics)

In [61]:
metrics_df_api

,svm,lof
f1_score,0.952836,0.915418
accuracy,0.910934,0.846194
roc_score,0.558785,0.538240


In [62]:
# List of columns you want to analyze
columns_to_analyze = ['ip_address']  # Replace with the actual column names

for column in columns_to_analyze:
    # Count the number of unique values and store them in a variable
    unique_values_count = df_api[column].nunique()

    # Get the count of each unique value and store it in a variable
    value_counts = df_api[column].value_counts()

    # Print the number of unique values for the current column
    print("Number of unique values in", column, ":", unique_values_count)

    # Iterate over the value_counts Series and print each unique value and its count
    for value, count in value_counts.items():
        print(value, ":", count)

    # Add a separator line between columns
    print('-' * 30)
print(len(df_api))

Number of unique values in ip_address : 2066
10.11.10.1 : 77868
10.11.10.2 : 462
10.11.26.129,10.11.10.1 : 26
10.11.12.146,10.11.10.1 : 26
10.11.28.201,10.11.10.1 : 26
10.11.28.34,10.11.10.1 : 26
10.11.28.99,10.11.10.1 : 26
10.11.23.29,10.11.10.1 : 26
10.11.23.88,10.11.10.1 : 24
10.11.22.68,10.11.10.1 : 24
10.11.26.177,10.11.10.1 : 23
10.11.27.166,10.11.10.1 : 23
10.11.12.33,10.11.10.1 : 22
10.11.22.178,10.11.10.1 : 22
10.11.22.143,10.11.10.1 : 22
10.11.24.100,10.11.10.1 : 22
10.11.24.15,10.11.10.1 : 22
10.11.26.23,10.11.10.1 : 22
10.11.27.252,10.11.10.1 : 22
10.11.28.80,10.11.10.1 : 22
10.11.21.243,10.11.10.1 : 22
10.11.26.18,10.11.10.1 : 22
10.11.22.241,10.11.10.1 : 22
10.11.28.49,10.11.10.1 : 22
10.11.26.253,10.11.10.1 : 22
10.11.23.223,10.11.10.1 : 22
10.11.24.48,10.11.10.1 : 22
10.11.25.55,10.11.10.1 : 21
10.11.22.188,10.11.10.1 : 21
10.11.22.191,10.11.10.1 : 21
10.11.21.132,10.11.10.1 : 21
10.11.22.254,10.11.10.1 : 21
10.11.24.231,10.11.10.1 : 21
10.11.27.210,10.11.10.1 : 21
10.1

10.11.26.57,10.11.10.1 : 11
10.11.27.107,10.11.10.1 : 11
10.11.26.46,10.11.10.1 : 11
10.11.25.253,10.11.10.1 : 11
10.11.22.177,10.11.10.1 : 11
10.11.25.134,10.11.10.1 : 11
10.11.26.76,10.11.10.1 : 11
10.11.23.232,10.11.10.1 : 11
10.11.23.46,10.11.10.1 : 11
10.11.22.125,10.11.10.1 : 11
10.11.23.230,10.11.10.1 : 11
10.11.26.249,10.11.10.1 : 11
10.11.22.217,10.11.10.1 : 11
10.11.22.112,10.11.10.1 : 11
10.11.12.1,10.11.10.1 : 11
10.11.26.113,10.11.10.1 : 11
10.11.27.108,10.11.10.1 : 11
10.11.28.142,10.11.10.1 : 11
10.11.12.120,10.11.10.1 : 11
10.11.24.238,10.11.10.1 : 11
10.11.22.105,10.11.10.1 : 11
10.11.24.52,10.11.10.1 : 11
10.11.11.253,10.11.10.1 : 11
10.11.25.70,10.11.10.1 : 11
10.11.28.152,10.11.10.1 : 11
10.11.27.128,10.11.10.1 : 11
10.11.26.17,10.11.10.1 : 11
10.11.24.60,10.11.10.1 : 11
10.11.27.150,10.11.10.1 : 11
10.11.12.43,10.11.10.1 : 11
10.11.22.231,10.11.10.1 : 11
10.11.21.218,10.11.10.1 : 11
10.11.24.79,10.11.10.1 : 11
10.11.25.32,10.11.10.1 : 11
10.11.22.96,10.11.10.1 : 11

10.11.25.233,10.11.10.1 : 7
10.11.21.152,10.11.10.1 : 7
10.11.26.196,10.11.10.1 : 7
10.11.27.222,10.11.10.1 : 7
10.11.25.37,10.11.10.1 : 7
10.11.28.105,10.11.10.1 : 7
10.11.21.149,10.11.10.1 : 7
10.11.25.197,10.11.10.1 : 7
10.11.28.128,10.11.10.1 : 7
10.11.22.247,10.11.10.1 : 7
10.11.27.199,10.11.10.1 : 7
10.11.21.167,10.11.10.1 : 7
10.11.23.65,10.11.10.1 : 7
10.11.12.113,10.11.10.1 : 7
10.11.22.199,10.11.10.1 : 7
10.11.24.44,10.11.10.1 : 7
10.11.21.185,10.11.10.1 : 7
10.11.12.153,10.11.10.1 : 7
10.11.23.155,10.11.10.1 : 7
10.11.21.160,10.11.10.1 : 7
10.11.26.205,10.11.10.1 : 7
10.11.26.33,10.11.10.1 : 7
10.11.23.6,10.11.10.1 : 7
10.11.27.136,10.11.10.1 : 7
10.11.25.1,10.11.10.1 : 7
10.11.21.201,10.11.10.1 : 7
10.11.27.191,10.11.10.1 : 7
10.11.23.72,10.11.10.1 : 7
10.11.12.160,10.11.10.1 : 7
10.11.24.157,10.11.10.1 : 7
10.11.28.193,10.11.10.1 : 7
10.11.22.223,10.11.10.1 : 7
10.11.21.138,10.11.10.1 : 7
10.11.12.51,10.11.10.1 : 7
10.11.28.177,10.11.10.1 : 7
10.11.27.68,10.11.10.1 : 7
10.